In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

tickers = ['DIS', "NFLX", "META", "NVDA", "AMD"]
functions = ["BALANCE_SHEET", "INCOME_STATEMENT", "CASH_FLOW", "EARNINGS", "TIME_SERIES_MONTHLY_ADJUSTED"][-1 : ]
API_KEY = "4AMBBI4OJ4X9XELB"
base_url = "https://www.alphavantage.co/query?function={API_FUNCTION}&symbol={ticker}" + "&apikey={}".format(API_KEY)

dfs = dict()
for ticker in tickers:
    dfs['ticker'] = ticker
    dfs['data'] = []
    for function in tqdm(functions):
        response = requests.get(base_url.format(**{'API_FUNCTION' : function, "ticker" : ticker}))
        if function == "TIME_SERIES_MONTHLY_ADJUSTED":
            df = pd.DataFrame(response.json()['Monthly Adjusted Time Series']).T
            df.columns = ['Open', "High", "Low", 'Close', 'Adj. Close', 'Volume', 'Dividends']
        elif function == 'EARNINGS':
            df = pd.DataFrame(response.json()['quarterlyEarnings'])
        else:
            df = pd.DataFrame(response.json()['quarterlyReports'])
        df.to_csv(f"{ticker}_{function}.csv")

In [13]:
from glob import glob

In [28]:
df = pd.DataFrame(columns = ['ticker', 'data_type', 'data'])
for file_ in glob('./../../New folder/*.csv'):
    ticker, *file_type = file_.split('\\')[-1].split('.')[0].split('_',)
    df = pd.concat([df, pd.DataFrame([{'ticker' : ticker, 'data_type' : '_'.join(file_type), 'data' : pd.read_csv(file_)}])], ignore_index = True)

In [40]:
df.data_type.unique()

array(['BALANCE_SHEET', 'CASH_FLOW', 'EARNINGS', 'INCOME_STATEMENT',
       'TIME_SERIES_MONTHLY_ADJUSTED'], dtype=object)

In [88]:
balance_sheet_df = df[df.data_type == "BALANCE_SHEET"]
tickers = pd.Series(balance_sheet_df.apply(lambda x: [x.ticker] * x.data.shape[0], axis = 1).sum())

balance_sheet_df = pd.concat(balance_sheet_df.data.tolist())
tickers.index = balance_sheet_df.index
tickers.name = 'ticker'
balance_sheet_df = pd.concat([balance_sheet_df, tickers], axis = 1)

balance_sheet_df

In [91]:
cash_flow_df = df[df.data_type == "CASH_FLOW"]
tickers = pd.Series(cash_flow_df.apply(lambda x: [x.ticker] * x.data.shape[0], axis = 1).sum())

cash_flow_df = pd.concat(cash_flow_df.data.tolist())
tickers.index = cash_flow_df.index
tickers.name = 'ticker'
cash_flow_df = pd.concat([cash_flow_df, tickers], axis = 1)

cash_flow_df

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,ticker
0,2024-03-31,USD,521000000,NaN,NaN,-636000000.0,3.520000e+08,1406000000,142000000,-338000000.0,...,NaN,NaN,NaN,NaN,1000000.0,NaN,NaN,NaN,123000000,AMD
1,2023-12-31,USD,381000000,NaN,NaN,-181000000.0,3.190000e+08,1240000000,139000000,320000000.0,...,NaN,NaN,NaN,NaN,-113000000.0,NaN,NaN,NaN,667000000,AMD
2,2023-09-30,USD,421000000,NaN,NaN,-547000000.0,7.650000e+08,-677000000,124000000,744000000.0,...,NaN,0.0,0.0,NaN,-507000000.0,NaN,NaN,NaN,299000000,AMD
3,2023-06-30,USD,379000000,NaN,NaN,236000000.0,1.788000e+09,693000000,125000000,100000000.0,...,NaN,0.0,0.0,NaN,0.0,NaN,1.600000e+07,NaN,27000000,AMD
4,2023-03-31,USD,486000000,NaN,NaN,73000000.0,-5.690000e+08,1805000000,158000000,86000000.0,...,NaN,NaN,NaN,NaN,-238000000.0,NaN,-1.010000e+09,NaN,-139000000,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2010-10-31,USD,212177000,NaN,NaN,45618000.0,5.265800e+07,46876000,21823000,-3374000.0,...,NaN,26279000.0,NaN,NaN,26279000.0,NaN,7.417100e+07,NaN,84862000,NVDA
54,2010-08-01,USD,34344000,NaN,NaN,-84308000.0,8.785600e+07,46573000,37644000,133818000.0,...,NaN,NaN,NaN,NaN,14800000.0,NaN,-7.025900e+07,NaN,-140961000,NVDA
55,2010-05-02,USD,-5397000,NaN,NaN,16822000.0,-2.118270e+08,47147000,17080000,-155011000.0,...,NaN,NaN,NaN,NaN,63100000.0,NaN,4.100000e+04,NaN,137594000,NVDA
56,2010-01-31,USD,69245000,NaN,NaN,22778000.0,-3.006500e+07,47914000,22575000,23008000.0,...,NaN,45837000.0,NaN,NaN,45837000.0,NaN,-1.672690e+08,NaN,131076000,NVDA


In [92]:
income_statement_df = df[df.data_type == "INCOME_STATEMENT"]
tickers = pd.Series(income_statement_df.apply(lambda x: [x.ticker] * x.data.shape[0], axis = 1).sum())

income_statement_df = pd.concat(income_statement_df.data.tolist())
tickers.index = income_statement_df.index
tickers.name = 'ticker'
income_statement_df = pd.concat([income_statement_df, tickers], axis = 1)

income_statement_df

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,ticker
0,2024-03-31,USD,2560000000,5473000000,4939000000,2913000000,36000000,620000000,1.525000e+09,3551000000,...,1406000000,71000000,-52000000,NaN,123000000.0,101000000.0,96000000,1502000000,123000000,AMD
1,2023-12-31,USD,2911000000,6452000000,5154000000,3257000000,342000000,644000000,1.511000e+09,3409000000,...,1253000000,370000000,-297000000,27000000.0,667000000.0,707000000.0,397000000,1650000000,667000000,AMD
2,2023-09-30,USD,2747000000,5774000000,4289000000,3053000000,224000000,576000000,1.507000e+09,2743000000,...,660000000,260000000,-39000000,26000000.0,299000000.0,281000000.0,286000000,946000000,299000000,AMD
3,2023-06-30,USD,2443000000,5331000000,4156000000,2916000000,-20000000,547000000,1.443000e+09,2683000000,...,693000000,4000000,-23000000,28000000.0,27000000.0,16000000.0,32000000,725000000,27000000,AMD
4,2023-03-31,USD,2359000000,5353000000,5384000000,2994000000,-145000000,585000000,1.411000e+09,3801000000,...,1805000000,-126000000,13000000,NaN,-139000000.0,-119000000.0,-101000000,2537000000,-139000000,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2009-07-26,USD,156723000,776520000,619797000,619797000,-110107000,73975000,1.928550e+08,266830000,...,49322000,-107101000,-1799000,NaN,NaN,-101563000.0,-110107000,-60785000,-105302000,NVDA
59,2009-04-26,USD,189696000,664231000,474535000,474535000,-230965000,118864000,3.017970e+08,420661000,...,50658000,-224821000,-23483000,NaN,NaN,NaN,-230965000,-180307000,-201338000,NVDA
60,2009-01-25,USD,141666000,481140000,339474000,339474000,-175083000,86440000,2.117790e+08,316749000,...,48055000,-170375000,-22710000,NaN,NaN,NaN,-175083000,-127028000,-147665000,NVDA
61,2008-10-26,USD,367843000,897655000,529812000,529812000,56796000,90349000,2.123600e+08,311047000,...,49304000,61003000,-745000,NaN,NaN,NaN,56796000,106100000,61748000,NVDA


In [107]:
earnings_df = df[df.data_type == "EARNINGS"]
tickers = pd.Series(earnings_df.apply(lambda x: [x.ticker] * x.data.shape[0], axis = 1).sum())

earnings_df = pd.concat(earnings_df.data.tolist())
tickers.index = earnings_df.index
tickers.name = 'ticker'
earnings_df = pd.concat([earnings_df, tickers], axis = 1)

earnings_df

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,ticker
0,2024-03-31,2024-04-30,0.62,0.61,0.01,1.6393,post-market,AMD
1,2023-12-31,2024-01-30,0.77,0.77,0.00,0.0000,post-market,AMD
2,2023-09-30,2023-10-31,0.70,0.68,0.02,2.9412,post-market,AMD
3,2023-06-30,2023-08-01,0.58,0.57,0.01,1.7544,post-market,AMD
4,2023-03-31,2023-05-02,0.60,0.56,0.04,7.1429,post-market,AMD
...,...,...,...,...,...,...,...,...
96,2000-04-30,2000-05-16,0.01,0.01,0.00,0.0000,pre-market,NVDA
97,2000-01-31,2000-02-15,0.01,0.01,0.00,0.0000,pre-market,NVDA
98,1999-10-31,1999-11-18,0.01,0.02,-0.01,-50.0000,pre-market,NVDA
99,1999-07-31,1999-08-18,0.02,0.02,0.00,0.0000,pre-market,NVDA


In [96]:
price_df = df[df.data_type == "TIME_SERIES_MONTHLY_ADJUSTED"]
tickers = pd.Series(price_df.apply(lambda x: [x.ticker] * x.data.shape[0], axis = 1).sum())

price_df = pd.concat(price_df.data.tolist())
tickers.index = price_df.index
tickers.name = 'ticker'
price_df = pd.concat([price_df, tickers], axis = 1)

price_df

,Open,High,Low,Close,Adj. Close,Volume,Dividends,ticker
0,148.11,173.14,141.1550,166.36,166.3600,885679305,0.0,AMD
1,180.10,187.24,145.2900,158.38,158.3800,1239990675,0.0,AMD
2,197.91,227.30,172.0000,180.49,180.4900,1523364836,0.0,AMD
3,169.27,193.00,161.8100,192.53,192.5300,1271828272,0.0,AMD
4,144.28,184.92,133.7413,167.69,167.6900,1954753974,0.0,AMD
...,...,...,...,...,...,...,...,...
289,83.72,110.40,57.0000,89.13,1.7036,13503600,0.0,NVDA
290,64.00,150.00,57.5000,84.48,1.6147,36567000,0.0,NVDA
291,37.13,68.25,35.7500,64.00,1.2232,19085100,0.0,NVDA
292,47.25,48.25,35.0000,37.06,0.7083,10294700,0.0,NVDA


In [110]:
balance_sheet_df = balance_sheet_df[['fiscalDateEnding', 'ticker', 'commonStockSharesOutstanding']]
cash_flow_df = cash_flow_df[['fiscalDateEnding', 'ticker', 'profitLoss', 'dividendPayoutCommonStock']]
income_statement_df = income_statement_df[['fiscalDateEnding', 'ticker', 'grossProfit', 'totalRevenue', 'ebitda', 'netIncome']]
earnings_df = earnings_df[['fiscalDateEnding', 'ticker', 'reportedEPS']]

In [503]:
temp1 = balance_sheet_df.set_index(['ticker', 'fiscalDateEnding']).merge(cash_flow_df.set_index(['ticker', 'fiscalDateEnding']), left_index = True, right_index = True, how = 'outer')

temp2 = income_statement_df.set_index(['ticker', 'fiscalDateEnding']).merge(earnings_df.set_index(['ticker', 'fiscalDateEnding']), left_index = True, right_index = True, how = 'outer')
final_data = temp1.merge(temp2, left_index = True, right_index = True)
del temp1 
del temp2

In [504]:
from datetime import datetime

In [505]:
import yfinance as yf

In [506]:
ticker = yf.Tickers(['DIS', "NFLX", "META", "NVDA", "AMD"])

In [507]:
price = ticker.history(period = 'max', interval = '3mo', start = "2000-01-01")

[*********************100%%**********************]  5 of 5 completed


In [508]:
price = price.stack(level = 1).reset_index()

In [509]:
from pandas.tseries.offsets import QuarterBegin, QuarterEnd

In [510]:
price['Date'] = price['Date'] + QuarterEnd(0)

In [511]:
price = price.groupby('Ticker').apply(
        lambda x: x.set_index('Date').resample('Q').agg({'Close' : 'last',
                                          'High': 'max','Low' : 'min', 'Open' : 'first',
                                          'Volume' : 'mean', 'Stock Splits' : 'sum', 'Dividends' : 'sum'}).reset_index()
).reset_index().drop(columns = ['level_1'])

In [512]:
final_data = final_data.reset_index()

In [513]:
final_data['fiscalDateEnding'] = (pd.to_datetime(final_data['fiscalDateEnding']) + QuarterEnd(0))

In [517]:
final_data = final_data.merge(price, left_on = ['ticker', 'fiscalDateEnding'], right_on = ['Ticker', 'Date'])

In [519]:
final_data.to_csv('final_data.csv', index = False)